This program trains MacBERTh to recognize which sentences are in the Geneva Bible and which are not. it takes in one CSV file containing both Geneva Bible sentences and non-Geneva Bible sentences written in Early Modern England
Author: Jerry Zou

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from datasets import Dataset as HFDataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# Load the CSV into a DataFrame
csvFilePath = "/Users/Jerry/Desktop/GenevaBibleRecognitionDataSetUpdated.csv"
#When using DCC:
# csvFilePath = "/hpc/group/datap2023ecbc/zz341/GenevaBibleRecognitionDataSetUpdated.csv"
# csvFilePath = "/hpc/group/datap2023ecbc/zz341/SmallDataGenevaBibleRecognitionDataSetUpdated.csv"
dataFrame = pd.read_csv(csvFilePath)

hf_dataset = HFDataset.from_pandas(dataFrame)
tokenizer = AutoTokenizer.from_pretrained('emanjavacas/MacBERTh')

def tokenize_function(examples):
    return tokenizer(examples['Text'], padding='max_length', truncation=True, max_length = 256)

tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.rename_column('Label', 'labels')
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

# Define the custom PyTorch Dataset class

class BibleDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        item = self.data[idx]
        return {key: val.clone().detach() if torch.is_tensor(val) else torch.tensor(val) for key, val in item.items()}

    def __len__(self):
        return len(self.data)

# Create the custom datasets
train_dataset = BibleDataset(train_dataset)
val_dataset = BibleDataset(val_dataset)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('emanjavacas/MacBERTh', num_labels=2)

# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Set up device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Training loop
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# Save the trained model and tokenizer
output_dir = '/Users/Jerry/Desktop'
# When using DCC:
#output_dir = "/hpc/group/datap2023ecbc/zz341"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Training complete. Model saved to", output_dir)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/44509 [00:00<?, ? examples/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emanjavacas/MacBERTh and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, o

  0%|          | 0/13353 [00:00<?, ?it/s]

KeyboardInterrupt: 